In [2]:
import requests
from pywikibot.data import api
import pywikibot
import pprint
import re 


In [3]:
def prettyPrint(variable):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(variable)

In [4]:
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository() 

In [5]:
#Incase multiple items are returned for the title, the second item in the query might be used to analyse which QID is a better fit

#For this if the end user, may give additional info which they hope will be part of description or aliases then right QID may be retrieved

In [6]:
def label_catch(query):
    labelmatch=[]

    for x in Data['query']['search']: 
        item = pywikibot.ItemPage(repo, x['title']) 
        wd_item=item.get()

        try:
            label=wd_item['labels']['en']
            if label==query[0]:
                labelmatch.append(x['title'])
        except:
            pass
        
    if len(labelmatch)>1:
        adtext=input("Many articles found with same label, addtional key words to check description or aliases?")
        query.append(adtext)
        returned=des_alais_check(query, labelmatch)
        return labelmatch 
    
        
    elif len(labelmatch)==0: 
        print("No match found in Wikidata Connected Pages")
        x=unconnected(query)
        if x==0:
            print("Match not found in Unconnected pages")

    elif len(labelmatch)==1: 
        print(labelmatch[0])
    return None

In [7]:
'''
query[0]-->article title/label

query[1]--> In the article, if a lot of QIDs are returned then this information will help to find a better fit by studying description and aliase

similar can be done for Statements or identifiers, infact specific terms for description and aliases can be separated as well
'''

def des_alais_check(query, labelmatch):


    for qid_check in labelmatch:

        item = pywikibot.ItemPage(repo, qid_check)
        wd_item=item.get()
        
        try:

            descriptions=wd_item['descriptions']['en']
            x=re.findall(query[1],descriptions,flags=re.IGNORECASE)
            #print(x)

            for value in x:
                #print(value)
                if value.lower()==query[1].lower():
                    print(f"Match: {query[0]}---{value}---{qid_check}")
                else:
                    print("Descriptions issue")
        except:
            pass

                #Alias check
        else:
            try:

                alias=wd_item['aliases']['en']

                for p in alias:
                    x=re.findall(query[1],p, flags=re.IGNORECASE)
                    #print(query[1])
                    #print(*x)

                    for value in x:
                        if value.lower()==query[1].lower():
                    
                            print(f"{query[0]}-{value}-{qid_check}")
                        else:
                            print("Aliase issue")
            except:
                pass
    return None 

In [10]:
text = input("Article Title/Label?")
query=[]
query.append(text)

api = "https://www.wikidata.org/w/api.php"
params = {
    'action': 'query',
    'list': 'search',
    'format': 'json',
    'srsearch': query[0],
    'srprop': 'snippet|titlesnippet',
    'srlimit': 100
}

r = requests.get(api, params=params)
Data=r.json()

label_match=label_catch(query)

Article Title/Label?Rama
Many articles found with same label, addtional key words to check description or aliases?Lord
Rama-Lord-Q160213


In [9]:
print(query)

['Narendra Modi']


In [41]:
print(label_match)

None


In [43]:
#For unconnected pages 
#https://bitbucket.org/mikepeel/wikicode/src/master/enwp_find_wikidata.py

def unconnected(query):
    enwp = pywikibot.Site('en', 'wikipedia')
    enwd = pywikibot.Site('wikidata', 'wikidata')
    targetcats = ['Category:Articles_without_Wikidata_item']

    for targetcat in targetcats:
        cat = pywikibot.Category(enwp, targetcat)
        pages = enwiki.querypage('UnconnectedPages')
        count=0
        for page in pages:
            if page.title()==query[0]:
                print(f"Match found in unconnected---->{page.title()}")
                count+=1
                
    return count